### Get Faces from Flickr by PUMA
This notebook does the following:
- Loads the shapefile for NYC PUMAs and uses PUMA centroid at geolocation for data collection
- USes Flicke API to get images for these geolocations
- Saves the images temporary in temp directory specified below
- Uses Dlib to filter images with faces
- Stores the filtered images in PUMA directory
- A directory of images is created for each of the PUMAs

In [ ]:
TEMP_DIR = 'temp/'
PUMA_DIR = 'pumaImages/'
pumaFile = '/home/urwa/Documents/Courses/Sem1/PUI/PUIData/Old Data/PUMA/geo_export_f0519f41-2898-4829-af71-3556b078f017.shp'

In [1]:
from __future__ import print_function
import os
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import requests 
import json
import numpy as np
import io
import json
import nltk
import shapely
from geopandas.tools import sjoin
from fiona.crs import from_epsg
import cv2
import urllib
import shutil
import os
import dlib

from flickrapi import FlickrAPI

In [ ]:
FLICKR_PUBLIC = '2aae62d210c499ff69e8a19daf975e98'
FLICKR_SECRET = 'f5e468b95ef21a55'

In [2]:
flickr = FlickrAPI(FLICKR_PUBLIC, FLICKR_SECRET, format='parsed-json')

In [3]:
def getPhotos(loc):
    extras = 'url_c'
    data = flickr.photos.search(lat =loc[0],lon=loc[1], radius = 5, per_page=500,extras=extras)
    photos = data['photos']['photo']
    return photos

In [4]:
def extractFaceImages(locName,loc):
    tempDir = TEMP_DIR
    processDir = PUMA_DIR
    targetDir = processDir+locName
    if not os.path.exists(targetDir):
        os.mkdir(targetDir)
    
    detector = dlib.get_frontal_face_detector()
    
    photos = getPhotos(loc)
    for p in photos:
        if 'url_c' in p:
            try :
                fname = p['url_c'].split('/')[-1]
                print(fname)
                urllib.urlretrieve(p['url_c'], tempDir+fname)
                img = dlib.load_rgb_image(tempDir+fname)
                dets = detector(img, 1)
                if len(dets)>0:
                    shutil.move(tempDir+fname, targetDir+"/"+fname)
                else:
                    os.remove(tempDir+fname)
            except:
                pass

In [6]:
pumashp = gpd.GeoDataFrame.from_file(pumaFile)
pumashp['centroid'] = pumashp.geometry.centroid
pumashp['lon'] = pumashp.centroid.apply(lambda p: p.x)
pumashp['lat'] = pumashp.centroid.apply(lambda p: p.y)
print(pumashp.shape)
pumashp.head(3)

(55, 7)


,puma,shape_area,shape_leng,geometry,centroid,lon,lat
0,3701,9.792852e+07,53227.144461,POLYGON ((-73.89641133483133 40.90450452082026...,POINT (-73.90488004491897 40.88944237744307),-73.904880,40.889442
1,3702,1.889860e+08,106050.002302,"POLYGON ((-73.8442314689986 40.86942035096838,...",POINT (-73.86164383659609 40.89000266817713),-73.861644,40.890003
2,3703,2.670137e+08,304071.257466,(POLYGON ((-73.78833349834532 40.8346671297593...,POINT (-73.81591891880724 40.85098182019305),-73.815919,40.850982


In [8]:
saveDf = pd.DataFrame(pumashp[['puma','lat','lon']])
saveDf.to_csv('pumaCentroid.csv',index=False)

In [12]:
Locations = zip(pumashp.puma,pumashp.lat,pumashp.lon)
for l in Locations:
    extractFaceImages(l[0],(l[1],l[2]))

47624657861_99808c0f32_c.jpg
40635139813_03468b5f5d_c.jpg
33716132798_b774e117fc_c.jpg
47539751612_2128f635ee_c.jpg
32617085747_d09d31802d_c.jpg
32609344767_cbdb050bea_c.jpg
47489380302_6c74e9cbac_c.jpg
47542153201_b70df5a356_c.jpg
40571537373_72cca151a5_c.jpg
40505904863_7c048867f1_c.jpg
46541038855_98dbdaea5a_c.jpg
46730169294_b2117ccb3e_c.jpg
47431334021_aff3641419_c.jpg
46427163745_ff8b1ddd03_c.jpg
46398329645_277ddcdeff_c.jpg
46537465854_0b71cda4ff_c.jpg
46346369675_d2c014707a_c.jpg
32319030207_cd96526ea9_c.jpg
47199544372_465d1266e3_c.jpg
33285704788_a73f086f7b_c.jpg
33285704758_94a5ac460d_c.jpg
47160856751_c0c65e408f_c.jpg
33285704108_9e4a7c3610_c.jpg
40195947963_ceb37c04ac_c.jpg
47055773342_30218b34eb_c.jpg
46308329714_a380761bd1_c.jpg
47032358101_7e7a623921_c.jpg
33140947768_7a3e559aa6_c.jpg
40016089473_b5889dc2bb_c.jpg
46980858311_718c984c9f_c.jpg
39947533803_29dcc9f969_c.jpg
46877288161_bcec734bef_c.jpg
31877858467_e1fdb7fd9a_c.jpg
46763604912_618ec0e6e9_c.jpg
31860079617_26

33682856928_0437ff15bd_c.jpg
46828164324_12874ef3f8_c.jpg
46828163924_df7152c7d0_c.jpg
33675109448_335a6b7edd_c.jpg
32609344767_cbdb050bea_c.jpg
32609344077_ae25ce93e0_c.jpg
47551739371_4cfd319776_c.jpg
47551738891_153ae844e0_c.jpg
40585730203_2bef5d2660_c.jpg
46827477204_29e6841f35_c.jpg
46827476934_a73bde49d1_c.jpg
40544554683_ffeb378508_c.jpg
46820216674_3198fd2f6b_c.jpg
47542155281_38652d547f_c.jpg
47542155231_9acbd5bea6_c.jpg
40576140313_d86ef37794_c.jpg
40576140283_90ba4b6667_c.jpg
40576140163_a818cecf1e_c.jpg
40576140033_bec3aea99f_c.jpg
47489380302_6c74e9cbac_c.jpg
47489380042_45afc6ee6a_c.jpg
47489379002_c878abbeea_c.jpg
47542153201_b70df5a356_c.jpg
47489361012_92f9504b06_c.jpg
40576105253_f512470bf3_c.jpg
40576104983_68951f837e_c.jpg
40576104603_5a5e56a764_c.jpg
47540431011_30287de2f2_c.jpg
46816707604_3e1a3df00c_c.jpg
33663558248_390bdb9640_c.jpg
47487516112_52a8bbc32e_c.jpg
46816687294_bb0b2a5b20_c.jpg
32597771707_0bbc851933_c.jpg
46816672824_818d33b92b_c.jpg
47540289431_a7

46828164324_12874ef3f8_c.jpg
33675109448_335a6b7edd_c.jpg
46828163704_4bb9137ed5_c.jpg
32609344767_cbdb050bea_c.jpg
32609344077_ae25ce93e0_c.jpg
47551739371_4cfd319776_c.jpg
47551738891_153ae844e0_c.jpg
40585730203_2bef5d2660_c.jpg
32608666787_7e0bc5e939_c.jpg
46827477204_29e6841f35_c.jpg
40544554683_ffeb378508_c.jpg
46820216674_3198fd2f6b_c.jpg
46819025334_8b2e7a3246_c.jpg
32597892697_6a57ef1a1c_c.jpg
32597892657_432bb2de47_c.jpg
46816803374_9bf10623a1_c.jpg
47540431681_4123cde07a_c.jpg
47540431631_f84dc31935_c.jpg
47540431561_87de2e7993_c.jpg
33663654638_2ab6840440_c.jpg
47540431511_3cf0ec1deb_c.jpg
33663654608_f62a2d2083_c.jpg
47540431421_f7873b44c8_c.jpg
33663654538_7da61d8aa9_c.jpg
46816802864_b63d6b444f_c.jpg
47540431011_30287de2f2_c.jpg
47540430841_dccf1cbbbf_c.jpg
33663579218_b71d67a84f_c.jpg
32597807247_29268e0bc4_c.jpg
32597807067_59db146278_c.jpg
33663579078_7db7ee8baa_c.jpg
33663558248_390bdb9640_c.jpg
47487516112_52a8bbc32e_c.jpg
32597771867_731662e314_c.jpg
46816687134_b0

47311059642_bac4b7d4f9_c.jpg
47311058392_f702bc55a7_c.jpg
33485989158_80c5a2bf2a_c.jpg
46622292144_dab572700c_c.jpg
46620818674_a291b2d684_c.jpg
40378861583_f0157e8603_c.jpg
47291429502_5c82388993_c.jpg
47291401312_b2106914b9_c.jpg
33468001388_f2824e973b_c.jpg
46429126225_e678ed08ef_c.jpg
46620767784_1728f777fc_c.jpg
46427163745_ff8b1ddd03_c.jpg
40364764733_bf3c7ba558_c.jpg
47277309332_e240a44216_c.jpg
46414973125_cb1fcf4f0a_c.jpg
46606720634_17329d9343_c.jpg
46606693954_eb7fdb5e15_c.jpg
46414831665_3486773143_c.jpg
46606598324_16cd3586c4_c.jpg
46606597824_100ea34560_c.jpg
46605759924_e8bfea07e4_c.jpg
32377277297_58b6a4ca5e_c.jpg
47260793472_1b79dccc08_c.jpg
46398329645_277ddcdeff_c.jpg
40347318823_79f0ef94a7_c.jpg
33436274898_80220bc72d_c.jpg
46397059295_0c48dd5aa5_c.jpg
33436274718_a8a73a3e7b_c.jpg
40342737683_c02ce073bf_c.jpg
32365773787_9e282b1936_c.jpg
47285218561_ac462506bd_c.jpg
40319369573_57dbd99929_c.jpg
46559927984_0f99759d17_c.jpg
46559927644_d3def145d6_c.jpg
46368543645_89

41394509404_f474955cb7_c.jpg
41169446225_675b803c39_c.jpg
41958788901_e040ef701a_c.jpg
40152234720_ec812daba6_c.jpg
41920304801_03a4762a50_c.jpg
26987171437_200f65a52b_c.jpg
40936189215_2931a2da1d_c.jpg
40936183925_8b4c986876_c.jpg
41792744432_5328e4b312_c.jpg
40001481790_fb2bc60e08_c.jpg
39963515290_8797d5be23_c.jpg
41765931271_18b5a65363_c.jpg
41630021152_bdea6ce1a1_c.jpg
39825526940_67fd3ba3c1_c.jpg
41593022172_5b2bebe7f0_c.jpg
41631686281_f76d3b170e_c.jpg
26634930067_39b7dd45f7_c.jpg
26583583477_2c32e58891_c.jpg
41308977532_2f7c5c7da8_c.jpg
41307321001_b82b057a1f_c.jpg
40256089045_328aa38b07_c.jpg
27180565008_96d1202990_c.jpg
27180201648_c06f585d8f_c.jpg
27180190208_6bf1ed5c03_c.jpg
41052864281_438e9f4c2a_c.jpg
40159597485_a351cd60d8_c.jpg
39243635110_071b983fd4_c.jpg
41052120951_6e375223cb_c.jpg
40064558655_c1ce777b43_c.jpg
40064554765_b40fd819cd_c.jpg
40770683362_65fbdb17c4_c.jpg
40812593931_b12913029e_c.jpg
39957288864_5b263176b7_c.jpg
39927773784_4d35395560_c.jpg
47365872721_af

...

42163368860_a777221fce_c.jpg
42158443880_fde3184f60_c.jpg
43248599174_6146fb6a7b_c.jpg
29029114207_721f9c66ff_c.jpg
42158428770_79c93373b9_c.jpg
43918748912_138e0a232a_c.jpg
42158426330_d20f84daa8_c.jpg
29029102107_c75cb25c69_c.jpg
42158414460_3a8db5a406_c.jpg
43918740812_179277bc4f_c.jpg
43803141342_680ffa0228_c.jpg
43726608802_303c03d133_c.jpg
43704322832_328d5139df_c.jpg
42846332465_e60ab9316c_c.jpg
42846331405_13ebf3b965_c.jpg
42975873474_6d60e5d1b0_c.jpg
43645344002_418a4fe69a_c.jpg
41883938720_397aee66c0_c.jpg
42975875044_4744ef98b6_c.jpg
42924969894_b230136436_c.jpg
41545638450_c9e739f227_c.jpg
43314796861_c7316dd27f_c.jpg
42596761624_7f323175af_c.jpg
43234070492_edb0b97a07_c.jpg
41419341680_4740c26f8e_c.jpg
42415999384_b6d5e87429_c.jpg
42315361534_c253fda3aa_c.jpg
28094834547_9679116910_c.jpg
28944035358_a69c3b388e_c.jpg
42782750221_2777e33771_c.jpg
40955778330_4d690f17db_c.jpg
27839317967_49b21780fc_c.jpg
42595645551_6ff901fb5d_c.jpg
27677113977_89d43f15f4_c.jpg
40719278260_c9

46414603851_f093f1921b_c.jpg
44597860390_717991975a_c.jpg
46401922911_2e6850617e_c.jpg
31462315707_81a9fa70c3_c.jpg
46350903082_5b1f6ec1a6_c.jpg
31461657147_2b2d80edd3_c.jpg
32500675218_82a60133cc_c.jpg
45623915864_9c9050df61_c.jpg
46296796982_7f6a9c83c5_c.jpg
31389584097_5003f7f2d2_c.jpg
46278221182_56139776f6_c.jpg
32456544518_2f6a6dd6e2_c.jpg
45605203974_2e4116f7b9_c.jpg
32454619448_e8590b95fc_c.jpg
32454619128_e2d3ea8cb9_c.jpg
46256762912_6cc09682c8_c.jpg
46235431302_c8d78316c9_c.jpg
46235426552_5a84055991_c.jpg
31346896477_0652711dd1_c.jpg
45561014124_55b95dbf22_c.jpg
45348924265_84e3e78b21_c.jpg
46237297541_0a4a992874_c.jpg
32361304778_59fd86032c_c.jpg
31294826087_19da1e8f54_c.jpg
46174273912_4d32e03558_c.jpg
45473375264_18851cea26_c.jpg
44362395840_d9eb20ed4e_c.jpg
32212283738_af4abb3f78_c.jpg
45139137105_dc56a48283_c.jpg
31112211597_c269238bda_c.jpg
45121385865_7036720b13_c.jpg
46021648501_b494ba9044_c.jpg
32145608388_c78a0552ef_c.jpg
32145607718_e09a876fa6_c.jpg
45986506421_2a

27318016664_ef9876d669_c.jpg
27723498442_7e66a35422_c.jpg
26890314411_8ec17aca44_c.jpg
26078173140_59a3d48f69_c.jpg
25986261096_ee689b2918_c.jpg
24738392816_c60ee2e279_c.jpg
24641394366_12de4b9be8_c.jpg
24634546515_318a7f95e1_c.jpg
23211946144_ae71dccb35_c.jpg
23749627746_f54e7b0f89_c.jpg
23285181749_c23f3ec45c_c.jpg
23544189712_29e0378044_c.jpg
23284677279_6f3a0d9e98_c.jpg
31599957830_0ec7261e68_c.jpg
31620723771_1228d22237_c.jpg
31695407726_00e60de9a2_c.jpg
30891499864_0c29b8881b_c.jpg
30877720073_4f2809c740_c.jpg
31245003501_3cccf8a680_c.jpg
31166341371_832596a102_c.jpg
31111646662_2e4a813a20_c.jpg
30415603153_75ff9ce3b8_c.jpg
30402009954_acac9fc72d_c.jpg
31187455486_7881fa4e23_c.jpg
30924831076_4bedc79e93_c.jpg
30112402284_1aeccd3278_c.jpg
30295349090_122ffede52_c.jpg
30314986592_97f9330494_c.jpg
30270879696_7bd1d5fbe7_c.jpg
29475076364_3161b8a425_c.jpg
29869099651_a6e1577ab5_c.jpg
29149606063_4049f6e230_c.jpg
29663860141_4f97a19ba2_c.jpg
28929343024_314d048b71_c.jpg
29222712650_bf

43889092015_3592732cf6_c.jpg
43889083595_a894800fa6_c.jpg
44799017721_b25f51ab80_c.jpg
43862755995_3b3dd48b4a_c.jpg
44722826182_e6cb4ab066_c.jpg
42939141810_7f12ecb93d_c.jpg
44699459592_922e188497_c.jpg
44736416261_975f8b2b6b_c.jpg
43827898875_35e67d6bc0_c.jpg
43827843085_7bb94d969d_c.jpg
43826218815_f29a9d43a7_c.jpg
29797943257_320a8524d5_c.jpg
30863473378_eb6e003238_c.jpg
44737057351_fc744456bc_c.jpg
43685045525_32185f7a5e_c.jpg
42784065240_e2b16a2777_c.jpg
44594262821_8b316405cf_c.jpg
44594259781_f3c4db1b79_c.jpg
42784061630_694a85d5b8_c.jpg
44594256341_0fa506938a_c.jpg
44544222212_72f7b06f66_c.jpg
43684667605_b576e13006_c.jpg
29656208047_bbf0fa6f6a_c.jpg
44544218742_f6b86b4811_c.jpg
44544217842_4e8a96e370_c.jpg
44594248891_21baefe6a0_c.jpg
44544217322_50c58df52a_c.jpg
44544216682_01ce9ce283_c.jpg
44575916341_a0cf3a4a67_c.jpg
30602988908_f02b696d53_c.jpg
43547214165_8426a554a1_c.jpg
42628576290_4030448ded_c.jpg
30559034798_304afff18b_c.jpg
44363791202_ff27140306_c.jpg
30503771448_0c

44279475655_219c21e5ff_c.jpg
31240398728_0b5b03763b_c.jpg
31043181938_99726b93b7_c.jpg
29791032037_445ef209fe_c.jpg
43818885365_e7fc32af46_c.jpg
43999245704_4df8e27b9c_c.jpg
44616999991_b7d8ecdcd5_c.jpg
43590856495_f42d60c102_c.jpg
43782655204_6365144f4f_c.jpg
29562672747_f1861d6ba7_c.jpg
29562672837_b07d816430_c.jpg
29562671077_767bb3d9e1_c.jpg
29562670887_9666796e0c_c.jpg
29510531107_62787a2283_c.jpg
42628212590_f8d1218686_c.jpg
29499554007_d58cc89448_c.jpg
43671859064_483ed94192_c.jpg
44344094531_d99e763bf1_c.jpg
43607741374_5ecc1b5c54_c.jpg
